In [1]:
#from bs4 import BeautifulSoup
import gcsfs  ## Google Cloud Storage file system
import os
import pandas as pd
import requests
import tarfile
import wfdb
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
from src.data.parsing import parse_txt

from src.data.gcp_pull_waveform_data import generate_record_map,generate_waveform_dataset

from src.data.bp_features import avg_bp,clean_bp_summary,merge_df

In [3]:
%env PROJECT_ID=qwiklabs-gcp-04-133e595cc3fe

env: PROJECT_ID=qwiklabs-gcp-04-133e595cc3fe


In [4]:
fs = gcsfs.GCSFileSystem(project=os.environ['PROJECT_ID'])
bucket = 'physionet_2009'
fs.ls(bucket)

['physionet_2009/mimic2cdb',
 'physionet_2009/test-set-a',
 'physionet_2009/test-set-b',
 'physionet_2009/train_wave']

In [5]:
# Get Train data Id

In [6]:
training_set = [
's21775', 's20658', 's22466','s05336', 's06349',
's08718', 's20794', 's24799', 's26318', 's14204',
's25699', 's07125', 's19208', 's12821', 's06637',
    
's02395', 's08779', 's23641', 's24924', 's00439', 
's23015', 's19603', 's02172', 's26105', 's23594',
's21817', 's24984', 's25602', 's23591', 's15687',
    
's17765', 's04860', 's26097', 's00318', 's14495',
's22888', 's26296', 's06180', 's07468', 's24004',
's03133', 's02280', 's09672', 's19418', 's15465',
    
's05786', 's25222', 's02561', 's24923', 's16019',
's04286', 's17069', 's07860', 's23020', 's24431',
's22657', 's09341', 's05126', 's10611', 's05289'
]

training_labels = [
'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1',
'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1',
    
'H2', 'H2', 'H2', 'H2', 'H2', 'H2', 'H2', 'H2',
'H2', 'H2', 'H2', 'H2', 'H2', 'H2', 'H2',
    
'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 
'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1',

'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2',
'C2', 'C2', 'C2', 'C2', 'C2', 'C2', 'C2'
]

len(training_set) == len(training_labels)

True

In [7]:
# Pull in records

In [8]:
record_map = generate_record_map(bucket, fs, training_set)

Dimensions of data set: (82, 6)
Data set reflects data for 60 clinical IDs
Data set reflects data for 82 waveform IDs


In [9]:
# turn into useful data frame

In [10]:
x = {e:generate_waveform_dataset(e, record_map) for e in record_map['waveform_entities'][0:3]}

{'Clinical': 's00318', 'Wave': 'a40802', 'Sex': 'M', 'Age': '58', 'Birthdate': '11/10/1950', 'Waveform': '14/07/2009'}
{'Clinical': 's00439', 'Wave': 'a40064', 'Sex': 'F', 'Age': '82', 'Birthdate': '22/12/1930', 'Waveform': '13/01/2013'}
{'Clinical': 's02172', 'Wave': 'a40099', 'Sex': 'M', 'Age': '32', 'Birthdate': '07/07/1982', 'Waveform': '20/10/2014'}


In [11]:
# calculate buckets

In [12]:
#avg_bp(x['a40802'], time_chunk = 60 * 5, time_window = 60,waveform_type = 'ABP')

bp_summary_outcome = {e: avg_bp(x[e], time_chunk = 60 * 5, time_window = 60,waveform_type = 'ABP') for e in x.keys()}
bp_summary_outcome = {e: clean_bp_summary(x) for e, x in bp_summary_outcome.items()}

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jupyter/waveform-fun/src/data/bp_features.py:52: RuntimeWarning: divide by zero encountered in true_divide
  peaks, _ = find_peaks(1 / vals)


In [13]:
bp_summary_inputs = {e: avg_bp(x[e], time_chunk = 60 * 20, time_window = 60,waveform_type = 'ABP') for e in x.keys()}
bp_summary_inputs = {e: clean_bp_summary(x) for e, x in bp_summary_inputs.items()}

In [14]:
final_df = pd.concat([merge_df(e_in,e_out,df_in,df_out) for (e_in, df_in),(e_out, df_out) in zip(bp_summary_inputs.items(), bp_summary_outcome.items())])

In [15]:
final_df
#df_out = bp_summary_outcome['a40802'] 
#df_in = bp_summary_inputs['a40802']

,start_window,end_window_outputs,start_window_time_outputs,end_window_time_outputs,avg_sys_outputs,avg_dias_outputs,avg_map_outputs,all_values_outputs,current_hypotensive_outputs,hypotensive_in_15_outputs,...,start_window_time_inputs,end_window_time_inputs,avg_sys_inputs,avg_dias_inputs,avg_map_inputs,all_values_inputs,current_hypotensive_inputs,hypotensive_in_15_inputs,include_in_model_inputs,waveform_id
0,0,37500,2009-07-14 07:15:59.592,2009-07-14 07:20:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,...,2009-07-14 07:15:59.592,2009-07-14 07:35:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,0,a40802
1,7500,45000,2009-07-14 07:16:59.592,2009-07-14 07:21:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,...,2009-07-14 07:16:59.592,2009-07-14 07:36:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,0,a40802
2,15000,52500,2009-07-14 07:17:59.592,2009-07-14 07:22:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,...,2009-07-14 07:17:59.592,2009-07-14 07:37:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,0,a40802
3,22500,60000,2009-07-14 07:18:59.592,2009-07-14 07:23:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,...,2009-07-14 07:18:59.592,2009-07-14 07:38:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,0,a40802
4,30000,67500,2009-07-14 07:19:59.592,2009-07-14 07:24:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,...,2009-07-14 07:19:59.592,2009-07-14 07:39:59.592,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0,0.0,0,a40802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,33352500,33390000,2014-10-23 20:22:30.704,2014-10-23 20:27:30.704,57.392796,42.081100,47.184998,"[44.8, 45.6, 45.6, 45.6, 45.6, 44.8, 44.0, 44....",1,1.0,...,2014-10-23 20:22:30.704,2014-10-23 20:42:30.704,43.020194,32.340802,35.900599,"[44.8, 45.6, 45.6, 45.6, 45.6, 44.8, 44.0, 44....",1,NaN,0,a40099
4448,33360000,33397500,2014-10-23 20:23:30.704,2014-10-23 20:28:30.704,57.317895,42.648421,47.538246,"[38.4, 39.2, 40.0, 40.8, 42.4, 43.2, 44.0, 44....",1,1.0,...,2014-10-23 20:23:30.704,2014-10-23 20:43:30.704,41.966009,31.790592,35.182398,"[38.4, 39.2, 40.0, 40.8, 42.4, 43.2, 44.0, 44....",1,NaN,0,a40099
4449,33367500,33405000,2014-10-23 20:24:30.704,2014-10-23 20:29:30.704,54.312014,40.913514,45.379680,"[40.0, 39.2, 39.2, 39.2, 39.2, 39.2, 39.2, 39....",1,1.0,...,2014-10-23 20:24:30.704,2014-10-23 20:44:30.704,40.454232,30.906820,34.089291,"[40.0, 39.2, 39.2, 39.2, 39.2, 39.2, 39.2, 39....",1,NaN,0,a40099
4450,33375000,33412500,2014-10-23 20:25:30.704,2014-10-23 20:30:30.704,49.925084,37.745819,41.805574,"[46.4, 47.2, 48.0, 48.8, 48.8, 49.6, 49.6, 49....",1,1.0,...,2014-10-23 20:25:30.704,2014-10-23 20:45:30.704,37.993097,29.129155,32.083803,"[46.4, 47.2, 48.0, 48.8, 48.8, 49.6, 49.6, 49....",1,NaN,0,a40099


In [16]:
#final_df = df_out.merge(right = df_in, on = 'start_window', suffixes = ['_outputs','_inputs'])